# Microsoft Foundry Hosted Agents: Build + Deploy from Notebook

This notebook demonstrates the **hosted agents** execution model (containerized, code-first) using the flow from the Microsoft Learn hosted agents page.

## What are Hosted Agents?

Unlike the classic prompt-based agents where Azure runs your agent logic, **hosted agents** let you package your own code as a Docker container that Azure deploys and scales. This gives you full control over your agent's runtime, dependencies, and behavior.

**Key differences from classic agents:**
- **Classic agents**: You define prompts/tools; Azure executes them in a managed runtime
- **Hosted agents**: You write and containerize your own agent code; Azure deploys your container

## What this notebook covers:

| Step | What | Why |
|------|------|-----|
| 1) Local test | Run hosting adapter on `localhost:8088` | Validate your agent works before containerizing |
| 2) Docker build | Create container image | Package your agent for deployment |
| 3) ACR push | Push image to Azure Container Registry | Azure needs to pull your image from ACR |
| 4) Capability host | Enable hosted agents on your account | One-time preview requirement |
| 5) Create version | Register container as hosted agent | Tell Foundry about your agent image |
| 6) Start deployment | Launch the hosted agent | Make it available for invocation |
| 7) Invoke | Call via Responses API | Use your running agent |

> **Prerequisites**
> - **Docker** installed and running
> - **Azure CLI** (`az`) installed and logged in
> - **Azure Container Registry (ACR)** created
> - **Azure AI Foundry project** in North Central US (preview limitation)
> - Appropriate **RBAC permissions** to assign roles

## 0) Prerequisites & Configuration

**Before running this notebook**, ensure you have:

1. **Environment variables** set in a `.env` file (or exported):
   - `AZURE_AI_PROJECT_ENDPOINT` or `PROJECT_ENDPOINT` — Your Foundry project endpoint URL
   - `MODEL_DEPLOYMENT_NAME` — The name of your deployed model (e.g., `gpt-4o`)
   - `ACR_NAME` — Your Azure Container Registry name (without `.azurecr.io`)

2. **Azure CLI logged in**: Run `az login` if not already authenticated

3. **Docker running**: The Docker daemon must be started

The cell below loads these values. If any are missing, you'll need to set them before proceeding to the deployment steps.

### ⚠️ Regional Availability — IMPORTANT

**Hosted agents are currently (30.12.2025) are supported only in North Central US.**

If your Foundry project is in any other region, you'll get the error:
> `"Hosted Agents are not enabled in this region."`

**What to do:**
1. **Create a new Foundry project in North Central US**, then run `create_version(...)` there.
2. **Keep your ACR accessible** to that project. ACR can technically be in another region, but cross-region adds latency and potential policy friction — keeping ACR in North Central US (or nearby) is cleaner.

This is a preview limitation and may expand to other regions in the future.

In [7]:
import os
from pathlib import Path

# Minimal config for the early (local) sections.
# Provide Azure/ACR variables when you reach the deploy sections.
APP_DIR = Path("hosted_agent_app")
PROJECT_ENDPOINT = os.getenv("AZURE_AI_PROJECT_ENDPOINT") or os.getenv("PROJECT_ENDPOINT") or ""
MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME") or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") or ""

print("APP_DIR:", APP_DIR)


APP_DIR: hosted_agent_app


## 1) Install Python Dependencies

**What this step does:**
Installs the required Python packages for building and deploying hosted agents.

**Packages installed:**
- `azure-ai-projects` — SDK for interacting with Azure AI Foundry projects, including creating hosted agent versions
- `azure-identity` — Azure authentication (DefaultAzureCredential)
- `python-dotenv` — Load environment variables from `.env` files
- `requests` — HTTP client for REST API calls
- `azure-ai-agentserver-core` / `azure-ai-agentserver-agentframework` — **Hosting adapter** packages that wrap your agent code into a Foundry-compatible HTTP service

**Why hosting adapters?**
The hosting adapter transforms your agent logic into a REST service that exposes the `/responses` endpoint. This is the contract Foundry expects from hosted agents — it sends requests to `/responses` and your agent returns responses in a compatible format.

In [8]:
# If you're on a clean env, uncomment.
# %pip install -U pip

# Azure AI Projects SDK (docs show a preview/beta in some sections)
# If you need the exact beta: %pip install --pre azure-ai-projects==2.0.0b2
%pip install -U azure-ai-projects azure-identity python-dotenv requests

# Hosting adapter packages (Python)
# These wrap your agent code into a Foundry-compatible HTTP service.
%pip install -U azure-ai-agentserver-core azure-ai-agentserver-agentframework


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2) Azure CLI Login + Sanity Checks

**What this step does:**
Verifies that your local environment is correctly configured before proceeding with Azure operations.

**Checks performed:**
1. **Azure subscription** — Confirms you're logged in and shows which subscription is active
2. **Azure CLI version** — Ensures you have a recent version (hosted agent commands require recent CLI)
3. **Docker version** — Confirms Docker is installed and accessible

**Why this matters:**
- If `az account show` fails → Run `az login` first
- If Docker fails → Start Docker Desktop or the Docker daemon
- Wrong subscription? → Run `az account set --subscription <name-or-id>`

**Alternative approach:**
This notebook uses `az` CLI + SDK. You could also use `azd` (Azure Developer CLI) for a more opinionated workflow, but the SDK approach shown here gives more visibility into each step.

In [9]:
import subprocess

def sh(cmd: str) -> str:
    """Run a shell command and return stdout. Raises on failure."""
    print(f"$ {cmd}")
    out = subprocess.check_output(cmd, shell=True, text=True)
    return out.strip()

print(sh("az account show --query name -o tsv"))
print(sh("az account show --query id -o tsv"))
print(sh("az version -o json | python3 -c \"import sys,json; print(json.load(sys.stdin)['azure-cli'])\""))
print(sh("docker --version"))

$ az account show --query name -o tsv
MCAPS-Hybrid-REQ-102171-2024-ozgurguler
$ az account show --query id -o tsv
a20bc194-9787-44ee-9c7f-7c3130e651b6
$ az version -o json | python3 -c "import sys,json; print(json.load(sys.stdin)['azure-cli'])"
2.81.0
$ docker --version
Docker version 29.1.3, build f52814d454


## 3) Write a Minimal Hosted-Agent App (Agent Framework + Hosting Adapter)

**What this step does:**
Creates a minimal Python agent application that can be hosted by Azure AI Foundry.

**Files created in `hosted_agent_app/`:**
- `agent_app.py` — Your agent code wrapped with the hosting adapter
- `.env` — Environment variables for local testing

**How the hosting adapter works:**
```
Your Agent Logic  →  Hosting Adapter  →  HTTP Server (:8088)  →  /responses endpoint
```

The adapter:
1. Starts an HTTP server (default port 8088)
2. Exposes a `/responses` endpoint that Foundry calls
3. Translates incoming requests to your agent function
4. Returns responses in Foundry's expected format

**Why this architecture?**
This is the same pattern whether you're testing locally or deploying to Azure. The only difference is:
- **Local**: You run the server directly with `python agent_app.py`
- **Azure**: Your container runs the same code, but Azure manages scaling and routing

**Customization:**
The skeleton `my_agent()` function is intentionally minimal. Replace it with your actual agent logic — you can use LangChain, LangGraph, Semantic Kernel, or raw OpenAI calls.

In [10]:
APP_DIR.mkdir(exist_ok=True)

# Minimal agent service app using the hosting adapter.
# IMPORTANT: this is a *template* because hosting adapter APIs can evolve.
# If you already have your agent-framework code, drop it into this directory.

(APP_DIR / "agent_app.py").write_text(
    '''\
"""Hosted Agent app (Python) - minimal skeleton.

This file is meant to run locally via hosting adapter on localhost:8088.
Later, it gets containerized for Foundry hosted agents.
"""

import os
from dotenv import load_dotenv

# NOTE: The exact import path can vary by package version.
# The hosting adapter concept is: wrap an agent into a Foundry-compatible HTTP service.
from azure.ai.agentserver.agentframework import from_agentframework

# Your agent logic can be as simple or complex as you want.
# Here we define an ultra-minimal "agent" callable.
def my_agent(user_text: str) -> str:
    return f"You said: {user_text}. (Hosted agent skeleton)"

def main():
    load_dotenv()
    # The adapter will start an HTTP server with Foundry-compatible endpoints.
    # Docs conceptually show one-line hosting like: from_langgraph(my_agent).run()
    # For Agent Framework wrapper we use from_agentframework.
    server = from_agentframework(my_agent)
    server.run(host="0.0.0.0", port=8088)

if __name__ == "__main__":
    main()
    ''',
    encoding="utf-8",
)

# .env for local run
(APP_DIR / ".env").write_text(
    f"""\
AZURE_AI_PROJECT_ENDPOINT={PROJECT_ENDPOINT}
MODEL_DEPLOYMENT_NAME={MODEL_DEPLOYMENT_NAME}
""",
    encoding="utf-8",
)

print("Wrote hosted agent app to:", APP_DIR)
print("Files:", [p.name for p in APP_DIR.iterdir()])


Wrote hosted agent app to: hosted_agent_app
Files: ['Dockerfile', 'agent_app.py', '.env']


## 4) Run Locally + Smoke Test `/responses`

**What this step does:**
Provides commands to test your agent locally before containerizing it.

**Why test locally first?**
- Faster iteration — no need to build/push Docker images
- Easier debugging — full access to logs and stack traces
- Validates the hosting adapter integration

**How to test:**

1. **Start the server** (in a separate terminal):
   ```bash
   cd hosted_agent_app && python agent_app.py
   ```

2. **Send a test request** (from another terminal):
   ```bash
   curl -s http://localhost:8088/responses \
     -H 'Content-Type: application/json' \
     -d '{"input": {"messages": [{"role": "user", "content": "Hello!"}]}}' | jq
   ```

**Expected behavior:**
- Server starts on port 8088
- `/responses` returns a JSON response with your agent's output
- If it works locally, it will work when containerized

**Troubleshooting:**
- Port in use? Change the port in `agent_app.py`
- Import errors? Check package installation
- Auth errors? The local server may need Azure credentials if your agent calls Azure services

In [11]:
# Start the local server in the background.
# In Jupyter, the simplest approach is to run it in a separate terminal.
# Here we print the command you should run.

print("Run in a terminal:")
print(f"cd {APP_DIR} && python agent_app.py")

print("Then test from another terminal:")
print("curl -s http://localhost:8088/responses -H 'Content-Type: application/json' \\")
print("  -d '{\"input\": {\"messages\": [{\"role\": \"user\", \"content\": \"Where is Seattle?\"}]}}' | jq")


Run in a terminal:
cd hosted_agent_app && python agent_app.py
Then test from another terminal:
curl -s http://localhost:8088/responses -H 'Content-Type: application/json' \
  -d '{"input": {"messages": [{"role": "user", "content": "Where is Seattle?"}]}}' | jq


In [12]:
# Image naming (used for local build and ACR push)
IMAGE_NAME = os.getenv("IMAGE_NAME") or "my-hosted-agent"
IMAGE_TAG = os.getenv("IMAGE_TAG") or "v1"
print("IMAGE:", f"{IMAGE_NAME}:{IMAGE_TAG}")


IMAGE: my-hosted-agent:v1


## 5) Create a Dockerfile + Build the Image

**What this step does:**
1. Creates a `Dockerfile` that packages your agent app into a container
2. Builds the Docker image locally

**Why containerize?**
Azure AI Foundry hosted agents run as containers. Containerization ensures:
- **Reproducibility** — Same environment locally and in Azure
- **Isolation** — Your dependencies don't conflict with others
- **Portability** — Deploy anywhere that runs containers

**Dockerfile breakdown:**
```dockerfile
FROM python:3.11-slim          # Base image with Python
WORKDIR /app                   # Set working directory
COPY . /app                    # Copy your agent code
RUN pip install ...            # Install dependencies
EXPOSE 8088                    # Declare the port (documentation)
CMD ["python", "agent_app.py"] # Start your agent server
```

**Image naming:**
- `IMAGE_NAME` — The name of your image (default: `my-hosted-agent`)
- `IMAGE_TAG` — Version tag (default: `v1`)
- Full local reference: `my-hosted-agent:v1`

**Tip:** Increment `IMAGE_TAG` (e.g., `v2`, `v3`) when you make changes to avoid caching issues.

In [13]:
# Minimal Dockerfile. Adjust if you have extra deps.
(APP_DIR / "Dockerfile").write_text(
"""\
FROM python:3.11-slim

WORKDIR /app

# Install runtime deps
COPY . /app

RUN pip install --no-cache-dir -U pip \
    && pip install --no-cache-dir -U python-dotenv azure-identity azure-ai-projects \
    && pip install --no-cache-dir -U azure-ai-agentserver-core azure-ai-agentserver-agentframework

EXPOSE 8088

CMD ["python", "agent_app.py"]
""",
    encoding="utf-8",
)

print("Dockerfile written.")
print("Building docker image...")
sh(f"docker build -t {IMAGE_NAME}:{IMAGE_TAG} {APP_DIR}")
print("Local image built:", f"{IMAGE_NAME}:{IMAGE_TAG}")


Dockerfile written.
Building docker image...
$ docker build -t my-hosted-agent:v1 hosted_agent_app


#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 378B done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.11-slim
#2 DONE 4.7s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 1.63kB done
#4 DONE 0.0s

#5 [1/4] FROM docker.io/library/python:3.11-slim@sha256:0b2cfd528ad7c43ba83de95e2788393cf545aae1ed738868e961454bd38a760a
#5 resolve docker.io/library/python:3.11-slim@sha256:0b2cfd528ad7c43ba83de95e2788393cf545aae1ed738868e961454bd38a760a done
#5 sha256:0b2cfd528ad7c43ba83de95e2788393cf545aae1ed738868e961454bd38a760a 10.37kB / 10.37kB done
#5 sha256:b8129af7ea9a0969030638589dae3dcaf77fb3a7c65ad20359c1d1a1bc2bf12e 1.75kB / 1.75kB done
#5 sha256:e1523b812f94df96c608086784fd00371e8b4902b18af6e39b311efabe62ec63 5.49kB / 5.49kB done
#5 sha256:2ae15a20160209c6fd6cff4886e4ba2e666fa5bedd7b

Local image built: my-hosted-agent:v1


#9 exporting layers 0.4s done
#9 writing image sha256:3822f28e355d58d63438f3efbdf153a51aaa2185023e08599e902e829bbc9ef2
#9 writing image sha256:3822f28e355d58d63438f3efbdf153a51aaa2185023e08599e902e829bbc9ef2 done
#9 naming to docker.io/library/my-hosted-agent:v1 done
#9 DONE 0.4s


## 6) Push Image to Azure Container Registry (ACR)

**What this step does:**
1. Configures the ACR connection using your `ACR_NAME`
2. Logs into ACR using Azure CLI credentials
3. Tags your local image with the ACR path
4. Pushes the image to ACR

**Why ACR?**
Azure AI Foundry can only pull container images from Azure Container Registry. When you create a hosted agent version, you provide an ACR image reference, and Foundry pulls from there.

**Image reference format:**
```
<acr-name>.azurecr.io/<image-name>:<tag>
Example: myregistry.azurecr.io/my-hosted-agent:v1
```

**Prerequisites:**
- `ACR_NAME` environment variable set (just the registry name, not the full `.azurecr.io` URL)
- You must have `AcrPush` role on the registry (or be an Owner/Contributor)

**What happens behind the scenes:**
1. `az acr login` — Gets a temporary Docker credential for your registry
2. `docker tag` — Creates an alias pointing to ACR
3. `docker push` — Uploads image layers to ACR

**Troubleshooting:**
- "unauthorized" → Check your Azure login and ACR permissions
- "not found" → Verify ACR_NAME is correct (no `.azurecr.io` suffix)

In [17]:
# Reload .env to pick up any changes
from dotenv import load_dotenv
load_dotenv(override=True)

# ACR config (required to push)
# NOTE: ACR_NAME should be just the registry name, e.g. "myregistry" (NOT "myregistry.azurecr.io")
ACR_NAME = os.getenv("ACR_NAME") or ""
if not ACR_NAME:
    raise ValueError("Set ACR_NAME in .env (e.g. ACR_NAME=myregistry, without .azurecr.io)")
ACR_LOGIN_SERVER = f"{ACR_NAME}.azurecr.io"
IMAGE_REF = f"{ACR_LOGIN_SERVER}/{IMAGE_NAME}:{IMAGE_TAG}"
print("IMAGE_REF:", IMAGE_REF)

IMAGE_REF: containervault01.azurecr.io/my-hosted-agent:v1


### Push the Container Image to ACR

**What this cell does:**
1. **Logs in to ACR** using `az acr login` (authenticates Docker with your registry)
2. **Tags the local image** with the ACR registry path
3. **Pushes the image** to ACR so Foundry can pull it later

**Prerequisites:**
- `ACR_NAME` must be set in your `.env` (just the registry name, e.g., `myregistry`)
- Docker must be running
- You must have `AcrPush` permissions on the registry

**What happens behind the scenes:**
```
Local: my-hosted-agent:v1
  ↓ docker tag
ACR:   myregistry.azurecr.io/my-hosted-agent:v1
  ↓ docker push
Azure: Image now available for Foundry to pull
```

In [18]:
# Log in to ACR and push the image
print("Logging into ACR...")
print(f"$ az acr login --name {ACR_NAME}")
sh(f"az acr login --name {ACR_NAME}")

print("Tagging + pushing...")
print(f"$ docker tag {IMAGE_NAME}:{IMAGE_TAG} {IMAGE_REF}")
sh(f"docker tag {IMAGE_NAME}:{IMAGE_TAG} {IMAGE_REF}")

print(f"$ docker push {IMAGE_REF}")
sh(f"docker push {IMAGE_REF}")

print(f"Pushed: {IMAGE_REF}")

Logging into ACR...
$ az acr login --name containervault01
$ az acr login --name containervault01
Tagging + pushing...
$ docker tag my-hosted-agent:v1 containervault01.azurecr.io/my-hosted-agent:v1
$ docker tag my-hosted-agent:v1 containervault01.azurecr.io/my-hosted-agent:v1
$ docker push containervault01.azurecr.io/my-hosted-agent:v1
$ docker push containervault01.azurecr.io/my-hosted-agent:v1
Pushed: containervault01.azurecr.io/my-hosted-agent:v1


## 7) Grant ACR Pull Permissions to the Foundry Project Managed Identity

**What this step does:**
Grants the Azure AI Foundry project's managed identity permission to pull images from your ACR.

**Why is this needed?**
When Foundry deploys your hosted agent, it needs to pull your container image from ACR. This requires:
1. The Foundry project has a **system-assigned managed identity**
2. That identity has **AcrPull** role on your ACR

**How to find the Principal ID:**
1. Go to Azure Portal → Your Foundry project resource
2. Navigate to **Identity** → **System assigned**
3. Copy the **Object (principal) ID**
4. Set it as `FOUNDRY_PROJECT_PRINCIPAL_ID` in your `.env`

**What the code does:**
```bash
az role assignment create \
  --assignee-object-id <principal-id> \
  --assignee-principal-type ServicePrincipal \
  --role 'AcrPull' \
  --scope <acr-resource-id>
```

**Common issues:**
- **"Principal not found"** → Double-check the principal ID from the portal
- **"Authorization failed"** → You need Owner or User Access Administrator role
- **Already assigned?** → The command is idempotent; re-running is safe

**Note:** This is a one-time setup per project/ACR combination.

In [19]:
# ===== YOU MUST SET THIS =====
# This is the managed identity principal object id for your Foundry project.
# In portal: Foundry project -> Identity -> System assigned -> Object (principal) ID
FOUNDRY_PROJECT_PRINCIPAL_ID = os.getenv("FOUNDRY_PROJECT_PRINCIPAL_ID") or ""
if not FOUNDRY_PROJECT_PRINCIPAL_ID:
    print("Set FOUNDRY_PROJECT_PRINCIPAL_ID env var before running this cell.")
else:
    # Get ACR resource ID
    acr_id = sh(f"az acr show -n {ACR_NAME} --query id -o tsv")
    print("ACR ID:", acr_id)

    # Assign pull permissions
    # Role name may vary; docs reference 'Container Registry Repository Reader'
    # If this role name isn't recognized, list ACR roles and pick the correct one.
    sh(
        f"az role assignment create --assignee-object-id {FOUNDRY_PROJECT_PRINCIPAL_ID} "
        f"--assignee-principal-type ServicePrincipal "
        f"--role 'AcrPull' --scope {acr_id}"
    )
    print("Assigned AcrPull to Foundry project identity.")


Set FOUNDRY_PROJECT_PRINCIPAL_ID env var before running this cell.


In [11]:
# Step 1: Get your subscription ID
print("Your subscription ID:")
print(sh("az account show --query id -o tsv"))

Your subscription ID:
$ az account show --query id -o tsv
a20bc194-9787-44ee-9c7f-7c3130e651b6


In [12]:
# Step 2: Find your Foundry account and resource group
# Look for the account that matches your PROJECT_ENDPOINT subdomain
print("Your AI Services accounts (find the one matching your PROJECT_ENDPOINT):\n")
print(sh("az cognitiveservices account list -o table"))
print("\n" + "="*80)
print("HOW TO USE:")
print("1. Look at your PROJECT_ENDPOINT (e.g., https://ACCOUNT_NAME.services.ai.azure.com/...)")
print("2. Find that ACCOUNT_NAME in the 'Name' column above")
print("3. Copy the corresponding 'ResourceGroup' value")
print("="*80)

Your AI Services accounts (find the one matching your PROJECT_ENDPOINT):

$ az cognitiveservices account list -o table
Kind            Location        Name                              ResourceGroup
--------------  --------------  --------------------------------  ---------------------
AIServices      eastus          agent-ai-servicesgezg             rg-openai
AIServices      eastus          agent-ai-servicesjq3h             rg-openai
AIServices      eastus          agent-ai-services7fxt             rg-openai
AIServices      eastus2         ai-ozgurgulerai5658070475260732   rg-ozgurgulerai
AIServices      eastus2         ai-eastus2hubozguler527669401205  rg-openai
AIServices      eastus2         ai-hubx611882637128               rg-openai
AIServices      eastus2         ai-hubxx118150369322              rg-ozgurguler-3950_ai
FormRecognizer  uksouth         docint-ozguler                    rg_xbip
AIServices      northcentralus  ozgur-m3q1pn4n-northcentralus     rg-openai
AIServices   

In [13]:
# Step 3: Extract values from your PROJECT_ENDPOINT automatically
import re
endpoint = PROJECT_ENDPOINT or os.getenv("AZURE_AI_PROJECT_ENDPOINT") or ""

if endpoint:
    # Extract account name from endpoint URL
    match = re.search(r'https://([^.]+)\.services\.ai\.azure\.com/api/projects/([^/]+)', endpoint)
    if match:
        account_name = match.group(1)
        project_name = match.group(2)
        print(f"Detected from your PROJECT_ENDPOINT:")
        print(f"  FOUNDRY_ACCOUNT_NAME = {account_name}")
        print(f"  FOUNDRY_PROJECT_NAME = {project_name}")
        print(f"\nAdd these to your .env file!")
    else:
        print("Could not parse endpoint. Set manually.")
else:
    print("PROJECT_ENDPOINT not set. Run cell 2 first, or set AZURE_AI_PROJECT_ENDPOINT in .env")

Detected from your PROJECT_ENDPOINT:
  FOUNDRY_ACCOUNT_NAME = ozgurguler-7212-resource
  FOUNDRY_PROJECT_NAME = ozgurguler-7212

Add these to your .env file!


## 8) Set Up Capability Host (Preview Requirement)

**What is a Capability Host?**
A Capability Host is a preview requirement that enables your Azure AI Foundry account to run hosted (containerized) agents. It's essentially a "feature flag" that tells Azure your account is configured for this capability.

**Why is it needed?**
- Hosted agents are in **preview** — not all accounts have this capability by default
- The capability host enables the **public hosting environment** where your containers run
- Without it, `create_version()` calls will fail with "Hosted Agents are not enabled"

**What the code does:**
1. **Gets an ARM (Azure Resource Manager) token** — This is your authentication to Azure's management API
2. **Calls a PUT request** to create/update the capability host resource on your Foundry account
3. **Enables public hosting** with `enablePublicHostingEnvironment: true`

**API endpoint structure:**
```
PUT https://management.azure.com/subscriptions/{sub}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{account}/capabilityHosts/{name}?api-version=2025-10-01-preview
```

**This is a one-time setup** per Foundry account (not per project). Once enabled, you can create multiple hosted agents.

**Common errors:**
- "Update not supported" → The capability host already exists; you can skip this step
- "Not found" → Check your subscription/resource group/account names
- 403/401 → You need Contributor role on the Foundry account

In [15]:
# Reload .env to pick up any changes you made
from dotenv import load_dotenv
load_dotenv(override=True)

# Azure resource context (required for management-plane + CLI operations)
SUBSCRIPTION_ID = os.getenv("AZ_SUBSCRIPTION_ID") or ""
RESOURCE_GROUP  = os.getenv("AZ_RESOURCE_GROUP") or ""
FOUNDRY_ACCOUNT_NAME = os.getenv("FOUNDRY_ACCOUNT_NAME") or ""
FOUNDRY_PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME") or ""

missing = [k for k, v in {
    "AZ_SUBSCRIPTION_ID": SUBSCRIPTION_ID,
    "AZ_RESOURCE_GROUP": RESOURCE_GROUP,
    "FOUNDRY_ACCOUNT_NAME": FOUNDRY_ACCOUNT_NAME,
    "FOUNDRY_PROJECT_NAME": FOUNDRY_PROJECT_NAME,
}.items() if not v]
if missing:
    raise ValueError("Missing required config for deploy sections: " + ", ".join(missing))

print("Azure context OK.")
print(f"  Subscription: {SUBSCRIPTION_ID}")
print(f"  Resource Group: {RESOURCE_GROUP}")
print(f"  Account: {FOUNDRY_ACCOUNT_NAME}")
print(f"  Project: {FOUNDRY_PROJECT_NAME}")

Azure context OK.
  Subscription: a20bc194-9787-44ee-9c7f-7c3130e651b6
  Resource Group: rg-ozgurguler-7212
  Account: ozgurguler-7212-resource
  Project: ozgurguler-7212


### Why do we need an ARM token and Capability Host?

**Capability Host** is a preview requirement for Foundry hosted agents. It tells Azure that your Foundry account is allowed to run containerized agents in a public hosting environment.

**What this cell does:**
1. **Gets an ARM token** — Azure Resource Manager (ARM) is the management plane for Azure. We need an access token to make API calls to create/update Azure resources.
2. **Calls the management API** — We `PUT` a `capabilityHost` resource on your Foundry account with `enablePublicHostingEnvironment=true`.

**Why it's needed:**
- Without this, the hosted agent deployment will fail because Azure doesn't know your account is configured for hosted agents.
- This is a one-time setup per Foundry account (not per project).

In [16]:
# Azure resource context (required for management-plane + CLI operations)
SUBSCRIPTION_ID = os.getenv("AZ_SUBSCRIPTION_ID") or ""
RESOURCE_GROUP  = os.getenv("AZ_RESOURCE_GROUP") or ""
FOUNDRY_ACCOUNT_NAME = os.getenv("FOUNDRY_ACCOUNT_NAME") or ""
FOUNDRY_PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME") or ""

missing = [k for k, v in {
    "AZ_SUBSCRIPTION_ID": SUBSCRIPTION_ID,
    "AZ_RESOURCE_GROUP": RESOURCE_GROUP,
    "FOUNDRY_ACCOUNT_NAME": FOUNDRY_ACCOUNT_NAME,
    "FOUNDRY_PROJECT_NAME": FOUNDRY_PROJECT_NAME,
}.items() if not v]
if missing:
    raise ValueError("Missing required config for deploy sections: " + ", ".join(missing))

print("Azure context OK.")


Azure context OK.


## 10) Start / Manage the Hosted Agent Deployment (Azure CLI)

**What this step does:**
Uses Azure CLI commands to manage your hosted agent's lifecycle.

**Available commands:**

| Command | Purpose |
|---------|---------|
| `az cognitiveservices agent start` | Start running a specific agent version |
| `az cognitiveservices agent stop` | Stop a running agent |
| `az cognitiveservices agent show` | Check current status (Running, Stopped, etc.) |
| `az cognitiveservices agent list-versions` | List all versions of an agent |
| `az cognitiveservices agent update` | Update deployment configuration |

**Required parameters:**
- `--account-name` — Your Foundry account name
- `--project-name` — Your Foundry project name  
- `--name` — Your hosted agent name
- `--agent-version` — Which version to start (e.g., "1", "2")

**Agent lifecycle:**
```
Created → Starting → Running → Stopping → Stopped
                ↑__________________________|
```

**Important notes:**
- Starting an agent may take 1-2 minutes as Azure provisions the container
- Only one version can be running at a time per agent name
- Check status with `az cognitiveservices agent show` before invoking

**Cost considerations:**
Running agents consume compute resources. Stop agents when not in use to avoid unnecessary charges.

In [ ]:
# Hosted agents SDK models require azure-ai-projects >= 1.0.0b11 (or 2.0.0b2 for some features)
# If you get ImportError, upgrade: pip install --pre azure-ai-projects>=1.0.0b11

try:
    from azure.identity import DefaultAzureCredential
    from azure.ai.projects import AIProjectClient
    from azure.ai.projects.models import (
        ImageBasedHostedAgentDefinition,
        ProtocolVersionRecord,
        AgentProtocol,
    )
except ImportError as e:
    print(f"Import error: {e}")
    print("\nHosted agent models not found in your SDK version.")
    print("Upgrade with: pip install --pre 'azure-ai-projects>=1.0.0b11'")
    raise

cred = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=cred)

# IMPORTANT:
# Your container might need env vars. Put only what your container expects.
env_vars = {
    "AZURE_AI_PROJECT_ENDPOINT": PROJECT_ENDPOINT,
    "MODEL_DEPLOYMENT_NAME": MODEL_DEPLOYMENT_NAME,
}

agent_version = client.agents.create_version(
    agent_name=HOSTED_AGENT_NAME,
    description="Hosted agent created from notebook",
    definition=ImageBasedHostedAgentDefinition(
        container_protocol_versions=[
            ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version=CONTAINER_PROTOCOL_VERSION)
        ],
        cpu=HOSTED_CPU,
        memory=HOSTED_MEMORY,
        image=IMAGE_REF,
        environment_variables=env_vars,
    ),
)

print("Created hosted agent version.")
print("Agent:", agent_version.name if hasattr(agent_version, "name") else HOSTED_AGENT_NAME)
print("Version object:", agent_version)

Created hosted agent version.
Agent: ozgur-hosted-agent
Version object: {'metadata': {}, 'object': 'agent.version', 'id': 'ozgur-hosted-agent:1', 'name': 'ozgur-hosted-agent', 'version': '1', 'description': 'Hosted agent created from notebook', 'created_at': 1767036678, 'definition': {'kind': 'hosted', 'container_protocol_versions': [{'protocol': 'responses', 'version': 'v1'}], 'cpu': '1', 'memory': '2Gi', 'environment_variables': {'AZURE_AI_PROJECT_ENDPOINT': 'https://ozgurguler-7212-resource.services.ai.azure.com/api/projects/ozgurguler-7212', 'MODEL_DEPLOYMENT_NAME': 'gpt-5-nano'}, 'image': 'containervault01.azurecr.io/my-hosted-agent:v1'}}


## 10) Start / Manage the Hosted Agent Deployment (Azure CLI)

Docs show `az cognitiveservices agent start/stop/update/list-versions/show`.

> You need the `agent-version` number. If the SDK object didn’t print it clearly, use `list-versions`.


In [ ]:
# Invoke the hosted agent via OpenAI Responses API
# Note: The hosted agent must be in "Running" state (check with list-versions or show commands)

try:
    from azure.ai.projects.models import AgentReference
except ImportError:
    print("AgentReference not available - upgrade azure-ai-projects")
    raise

openai_client = client.get_openai_client()

# Create the agent reference directly (no need to retrieve first)
agent_ref = AgentReference(name=HOSTED_AGENT_NAME, version=AGENT_VERSION_TO_START)

resp = openai_client.responses.create(
    input=[{"role": "user", "content": "Write a haiku about Azure AI Foundry hosted agents."}],
    extra_body={"agent": agent_ref.as_dict()},
)

print(resp.output_text)

NameError: name 'AGENT_VERSION_TO_START' is not defined

In [ ]:
# ===== SET THE VERSION YOU WANT TO START =====
AGENT_VERSION_TO_START = os.getenv("AGENT_VERSION_TO_START") or "1"

print(sh(
    f"az cognitiveservices agent start "
    f"--account-name {FOUNDRY_ACCOUNT_NAME} "
    f"--project-name {FOUNDRY_PROJECT_NAME} "
    f"--name {HOSTED_AGENT_NAME} "
    f"--agent-version {AGENT_VERSION_TO_START} "
    f"-o json"
))


In [ ]:
# Invoke the hosted agent via OpenAI Responses API
# Note: The hosted agent must be in "Running" state (check with az cognitiveservices agent show)

try:
    from azure.ai.projects.models import AgentReference
except ImportError:
    print("AgentReference not available - upgrade azure-ai-projects")
    raise

openai_client = client.get_openai_client()

# Create the agent reference directly (no need to retrieve the agent first)
agent_ref = AgentReference(name=HOSTED_AGENT_NAME, version=AGENT_VERSION_TO_START)

resp = openai_client.responses.create(
    input=[{"role": "user", "content": "Write a haiku about Azure AI Foundry hosted agents."}],
    extra_body={"agent": agent_ref.as_dict()},
)

print(resp.output_text)

## 12) (Optional) Classic Prompt-Based Agent for Comparison

**Why include this section?**
To illustrate the key difference between hosted agents and classic agents.

**Classic agents (AgentsClient):**
- You define the agent using prompts, instructions, and tool configurations
- Azure Foundry executes your agent definition in its managed runtime
- You don't control the runtime environment or dependencies
- Great for simple agents that use built-in tools (Code Interpreter, File Search, etc.)

**Hosted agents (this notebook):**
- You write your own agent code in Python (or any language)
- You containerize it and deploy it to Azure
- You have full control over dependencies, frameworks, and behavior
- Great for complex agents, custom logic, or specific framework requirements (LangChain, Semantic Kernel, etc.)

**When to use which?**

| Use Case | Recommended Approach |
|----------|---------------------|
| Simple Q&A with built-in tools | Classic agent |
| Custom business logic | Hosted agent |
| Third-party API integrations | Hosted agent |
| LangChain/LangGraph agents | Hosted agent |
| Quick prototyping | Classic agent |
| Production with custom scaling | Hosted agent |

**The code below** shows how you would create a classic agent for comparison. It's commented out but can be run if you want to see both approaches side by side.

## 12) (Optional) Keep your classic prompt-based agent client for comparison

This is your original approach (agent definition executed inside the classic Foundry agent runtime). It does **not** deploy code.

Hosted agents = your code deployed as a container.


In [ ]:
## 13) Cleanup

**When you're done testing**, remember to clean up resources to avoid unnecessary charges.

**Hosted agent cleanup commands:**

```bash
# Stop the running agent (stops billing for compute)
az cognitiveservices agent stop \
  --account-name <account> \
  --project-name <project> \
  --name <agent-name>

# Delete a specific deployment
az cognitiveservices agent delete-deployment \
  --account-name <account> \
  --project-name <project> \
  --name <agent-name>

# Delete the agent entirely (all versions)
az cognitiveservices agent delete \
  --account-name <account> \
  --project-name <project> \
  --name <agent-name>
```

**Optional: Clean up ACR images**
```bash
# Delete a specific tag
az acr repository delete --name <acr-name> --image <image>:<tag>

# Or delete the whole repository
az acr repository delete --name <acr-name> --repository <image>
```

**What to keep vs. delete:**
- **Keep**: ACR (you'll reuse it), Foundry project, Capability Host (one-time setup)
- **Delete**: Old agent versions you no longer need, stopped deployments

**Cost considerations:**
- Stopped agents don't incur compute costs
- ACR storage has minimal costs for small images
- The Foundry project itself has no idle cost

## 13) Cleanup

To stop / delete the hosted deployment:

- `az cognitiveservices agent stop ...`
- `az cognitiveservices agent delete-deployment ...`
- `az cognitiveservices agent delete ...`

And optionally delete ACR image/tag.
